In [1]:
#Lakshminarayanan
import pprint
import pandas as pd
import numpy as np
import pickle
import joblib

# Loading processed_corpus from pickle

**processed_pickle has tokenized value of song_data_with_genre['text'], which as given below**

song_data_with_genre['text'] = song_data_with_genre['title'] + ' ' + song_data_with_genre['artist_name'] + ' ' + song_data_with_genre['genre'] + ' ' + song_data_with_genre['release']

In [2]:
processed_corpus = pickle.load( open( "processed_corpus.pkl", "rb" ) )

# Using corpora dictionary from gensim 

In [3]:
from gensim import corpora

dictionary = corpora.Dictionary(processed_corpus)

In [4]:
len(dictionary.token2id)

43757

In [5]:
# testing with new doc
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

[(3004, 1), (7344, 1)]


# USING SparseMatrixSimilarity FROM GENSIM

In [9]:
tfidf = joblib.load('gensim_tfidf_model.pkl')

In [10]:
index = joblib.load('gensim_sparse_similarities.pkl')

In [11]:
query_document = "Elton John".lower().split()
query_bow = dictionary.doc2bow(query_document)
sims = index[tfidf[query_bow]]

In [12]:
sims[:10]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [13]:
doc_l = []
score_l = []
for document_number, score in sorted(enumerate(sims), key=lambda x: x[1], reverse=True):
    doc_l.append(document_number)
    score_l.append(score)
    

In [14]:
recommended_songs = pd.DataFrame(score_l,doc_l).head(10)
recommended_songs=recommended_songs.reset_index()
recommended_songs.columns = ['index','score']
recommended_songs.set_index('index',inplace = True)
recommended_songs

,score
index,
53092,0.917258
159502,0.901199
97122,0.869411
157010,0.864635
86828,0.794120
61951,0.741077
53753,0.668568
7438,0.638758
98942,0.632582


In [15]:
len(sims)

183885

In [16]:
sims[document_number]

0.0

In [17]:
song_data_with_genre=pd.read_csv('song_data_with_genre.csv')

In [18]:
#song_data_with_genre = song_data_with_genre.drop(columns=['Unnamed: 0','Unnamed: 0.1','tid'])

In [19]:
song_data_with_genre

,Unnamed: 0,title,artist_name,genre,song_id,release,year,artist_gender,text
0,0,El hijo del pueblo,Jorge Negrete,Latin,SOLJTLX12AB01890ED,32 Grandes Éxitos CD 2,1997,female,El hijo del pueblo Jorge Negrete Latin 32 Gran...
1,1,N Gana,Waldemar Bastos,World,SOGPCJI12A8C13CCA0,Afropea 3 - Telling Stories To The Sea,0,male,N Gana Waldemar Bastos World Afropea 3 - Telli...
2,2,006,Lena Philipsson,Pop,SOSDCFG12AB0184647,Lena 20 År,1998,male,006 Lena Philipsson Pop Lena 20 År
3,3,Rock-N-Rule,Emery,Rock,SOIMMJJ12AF72AD643,I'm Only A Man (Bonus Track Version),2007,female,Rock-N-Rule Emery Rock I'm Only A Man (Bonus T...
4,4,I'm Ready,Aerosmith,Rock,SOEEHEY12CF5F88FB4,Honkin' On Bobo,2004,female,I'm Ready Aerosmith Rock Honkin' On Bobo
...,...,...,...,...,...,...,...,...,...
183880,185179,Interrotte Speranze,Coronatus,Metal,SOIQNVK12A8C13B061,Lux Noctis,2007,male,Interrotte Speranze Coronatus Metal Lux Noctis
183881,185180,Wouldn't Get Far,The Game,Rap,SOQDFGV12AB0183F25,R&B Collaborations 2007,0,female,Wouldn't Get Far The Game Rap R&B Collaboratio...
183882,185181,Wouldn't Get Far,The Game,Rap,SOYKHSL12AF72A35AC,Wouldn't Get Far,0,female,Wouldn't Get Far The Game Rap Wouldn't Get Far
183883,185182,Chill,Jazz Addixx,Rap,SOIDAMG12AB0182B3A,Oxygen Refreshed (Instrumentals),0,female,Chill Jazz Addixx Rap Oxygen Refreshed (Instru...


In [20]:
song_data_with_genre.duplicated(subset=['song_id']).value_counts()

False    183885
dtype: int64

In [21]:
#song_data_with_genre.drop_duplicates(subset=['song_id'])

In [22]:
d = []
for idx in recommended_songs.index:
    dic = dict(song_data_with_genre.loc[idx][['title','artist_name']])
    d.append(dic)

In [23]:
final_recommendation_df = pd.DataFrame(d)
final_recommendation_df

,title,artist_name
0,The Cage,Elton John
1,I Need You To Turn To,Elton John
2,First Episode At Hienton,Elton John
3,Grey Seal,Elton John
4,Philadelphia Freedom,Elton John
5,Chime Bells,Elton Britt
6,Just Like Belgium,Elton John
7,Believe,Elton John
8,Heart In The Right Place,Elton John
9,In Neon,Elton John


In [24]:
final_recommendation_df['score'] = list(recommended_songs.score.values)

In [25]:
final_recommendation_df

,title,artist_name,score
0,The Cage,Elton John,0.917258
1,I Need You To Turn To,Elton John,0.901199
2,First Episode At Hienton,Elton John,0.869411
3,Grey Seal,Elton John,0.864635
4,Philadelphia Freedom,Elton John,0.794120
5,Chime Bells,Elton Britt,0.741077
6,Just Like Belgium,Elton John,0.668568
7,Believe,Elton John,0.638758
8,Heart In The Right Place,Elton John,0.632582
9,In Neon,Elton John,0.621983
